In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    log_loss,
    roc_auc_score,
    recall_score,
    precision_score,
    accuracy_score,
    plot_roc_curve,
    plot_confusion_matrix,
    roc_curve,
    confusion_matrix,
)
import itertools
from tensorflow.keras.initializers import Constant, TruncatedNormal
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from numpy import mean, absolute

# Oversampling and under sampling
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

Learning Bias Mitigation: Parental Education Background <> KNN

In [ ]:
## define count of n from temporal models
n = list(range(2, 61))

Models

In [ ]:
## define metrics dataframe
metrics = pd.DataFrame(
    columns=[
        "model",
        "group",
        "subgroup",
        "Length",
        "Sentence",
        "Accuracy",
        "n_neighbors",
        "weights",
        "Precision",
        "Recall",
        "AUC",
        "FPR",
    ]
)

In [ ]:
## define feature cols
feature_cols = [
    "Erstloesung",
    "Schussel",
    "Erfolg",
    "Schwierigkeit",
    "ist_Schulzeit",
    "MehrfachFalsch",
    "vorher_abgebrochen",
    "Fehler",
    "Klassenstufe",
    "Jahredabei",
    "AnzahlAufgaben",
    "Sex__m",
    "Sex__w",
    "Testposition__pruefung",
    "Testposition__training",
    "Testposition__version",
    "Art__GK",
    "Art__GR",
    "Art__GZ",
    "Art__K",
    "Art__LB",
    "UserAttribut",
    "OrderNumber",
    "steps",
]

In [ ]:
"""
calculate and extract relevant metrics from y and pred
return metrics
"""


def get_metrics(clf, X, y, cv, pred):
    a = accuracy_score(y, pred)
    p = precision_score(y, pred)
    r = recall_score(y, pred)

    roc_auc = roc_auc_score(y, pred)
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    fpr = fp / (fp + tn)

    return a, p, r, roc_auc, fpr

In [ ]:
n_neighbors = [2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 3, 4, 5, 6, 7, 8, 9, 10]
weights = [
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
]

## build model for different parameters
for n_neighbors, weights in zip(n_neighbors, weights):
    for i in n:
        path = "AbiEltern_allsessions/matrix" + str(i) + ".pkl"
        infile = open(path, "rb")
        df = pickle.load(infile)
        infile.close()
        df = df.reset_index()
        X = df[feature_cols]
        y = df.y
        y = y.astype("int")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=1
        )
        k = 5
        cv = KFold(n_splits=k, random_state=None)

        # fit
        knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
        knn = knn.fit(X_train, y_train)
        pred = knn.predict(X_test)

        a, p, r, roc_auc, fpr = get_metrics(knn, X_test, y_test, cv, pred)
        metrics = metrics.append(
            {
                "model": "KNN",
                "group": "all",
                "subgroup": "all",
                "Length": len(df),
                "Sentence": i,
                "Accuracy": a,
                "n_neighbors": n_neighbors,
                "weights": weights,
                "Precision": p,
                "Recall": r,
                "AUC": roc_auc,
                "FPR": fpr,
            },
            ignore_index=True,
        )

        group = ["abiEltern", "abiEltern"]
        subgroup = ["abi", "keinAbi"]
        matrice = ["matrices_forte_abi", "matrices_forte_keinAbi"]

        for group, subgroup, matrix in zip(group, subgroup, matrice):
            path = (
                "../../02_dropout_prediction/01_keep_it_up/"
                + matrix
                + "/matrix"
                + str(i)
                + ".pkl"
            )
            infile = open(path, "rb")
            df = pickle.load(infile)
            infile.close()
            df = df.reset_index()
            X = df[feature_cols]
            y = df.y
            y = y.astype("int")
            pred = knn.predict(X)

            a, p, r, roc_auc, fpr = get_metrics(knn, X, y, cv, pred)
            metrics = metrics.append(
                {
                    "model": "KNN",
                    "group": group,
                    "subgroup": subgroup,
                    "Length": len(df),
                    "Sentence": i,
                    "Accuracy": a,
                    "n_neighbors": n_neighbors,
                    "weights": weights,
                    "Precision": p,
                    "Recall": r,
                    "AUC": roc_auc,
                    "FPR": fpr,
                },
                ignore_index=True,
            )

Evaluate

In [ ]:
## construct dfs from metric df
grouped = metrics.groupby(df.group)
grouped = df.groupby(df.group)
df_abiEltern = grouped.get_group("abiEltern")

df_abiEltern = df_abiEltern.drop(columns=["group", "Accuracy", "model"])
df_abiEltern = pd.pivot_table(
    df_abiEltern,
    values=["Precision", "Recall", "AUC", "FPR"],
    index=["n_neighbors", "weights", "Sentence"],
    columns=["subgroup"],
)
df_abiEltern["PP"] = df_abiEltern.Precision.abi - df_abiEltern.Precision.keinAbi
df_abiEltern["EO"] = df_abiEltern.Recall.keinAbi - df_abiEltern.Recall.abi
df_abiEltern["SA"] = df_abiEltern.AUC.abi - df_abiEltern.AUC.keinAbi
df_abiEltern["PE"] = df_abiEltern.FPR.keinAbi - df_abiEltern.FPR.abi
df_abiEltern = df_abiEltern.drop(columns=["AUC", "Precision", "Recall", "FPR"])
df_abiEltern.columns = df_abiEltern.columns.droplevel(1)
df_abiEltern = pd.pivot_table(
    df_abiEltern,
    values=["PP", "EO", "SA", "PE"],
    index=["Sentence"],
    columns=["n_neighbors", "weights"],
)

In [ ]:
## claculate mean of the results and map in data frame
met = ["EO", "PE", "PP", "SA"]
fertig = pd.DataFrame()
n_neighbors = [2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 3, 4, 5, 6, 7, 8, 9, 10]
weights = [
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "uniform",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
    "distance",
]

for n, w in zip(n_neighbors, weights):
    for x in met:
        for r in [(2, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60)]:
            values = np.mean(df_abiEltern[x][n][w][r[0] : r[1]])
            temp = pd.DataFrame(
                {
                    "Metrik": [x],
                    "Model": "KNN",
                    "Range": "{}-{}".format(r[0], r[1] - 1),
                    "Val": values,
                    "n_neighbors": n,
                    "weights": w,
                }
            )
            fertig = fertig.append(temp, ignore_index=True)

In [ ]:
"""
functions to format results
set two threshols: one at |0.02| in orange and one at |0.05| in red
format all negative values in bold
"""


def threshold001(v, props=""):
    return props if (v > 0.02) or (v < -0.02) else None


def threshold005(v, props=""):
    return props if (v > 0.05) or (v < -0.05) else None


def negativeValue(v, props=""):
    return props if (v < 0) else None


def showTable(df):
    styled = (
        df.style.set_properties(color="black", align="right")
        .set_properties(**{"background-color": "white"})
        .applymap(threshold001, props="color:orange;")
        .applymap(threshold005, props="color:red;")
        .applymap(negativeValue, props="font-weight:bold;")
    )
    return styled

In [ ]:
## show results table
mean_table = pd.pivot_table(
    fertig,
    values=["Val"],
    index=["n_neighbors", "weights", "Range"],
    columns=["Metrik", "Model"],
)
showTable(mean_table)

In [ ]:
## code to print results of specific measures
# grouped = metrics.groupby(metrics.group)
# df_all = grouped.get_group("all")
# modell = df_all.groupby(df_all.n_neighbors)
# five = modell.get_group(2)
# n = five.groupby(five.weights)
# f = n.get_group("uniform")
# ax = sns.lineplot(data=f, x='Sentence', y='Accuracy', hue='model')